In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import pylab
import calendar
import seaborn as sn
from scipy import stats
#import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [2]:
#dataset
df = pd.read_csv('csv_files/toll_weather.csv')
print(df.ftypes)
df.head()

/home/slittle/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Unnamed: 0          int64:dense
EndZoneID           int64:dense
StartZoneID         int64:dense
StartDateTime      object:dense
EndDateTime        object:dense
TollRate          float64:dense
Direction          object:dense
Temperature       float64:dense
Pressure          float64:dense
Humidity          float64:dense
Wind Direction     object:dense
Wind Speed        float64:dense
Weather Desc       object:dense
Cloud Cover        object:dense
Visibility        float64:dense
dtype: object


,Unnamed: 0,EndZoneID,StartZoneID,StartDateTime,EndDateTime,TollRate,Direction,Temperature,Pressure,Humidity,Wind Direction,Wind Speed,Weather Desc,Cloud Cover,Visibility
0,1940034,3120,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
1,1940033,3130,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
2,1940289,3230,3210,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,W,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
3,1940035,3110,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0
4,1940036,3100,3100,2018-06-07 15:18:00+00:00,2018-06-07 15:24:00+00:00,0.0,E,24.0,763.6,57.0,Wind blowing from the south-southeast,4.0,NaN,"Scattered clouds (40-50%) 1650 m, broken cloud...",16.0


In [3]:
df['Weather'] = df['Weather Desc'].map({'Thunderstorm':'heavy', 
 'Light thunderstorm, rain':'med', 'Heavy thunderstorm, rain':'heavy',
 'Mist':'light', 'In the vicinity thunderstorm':'light',
 'In the vicinity thunderstorm, light rain':'light', 'Light rain, mist':'light',
 'Thunderstorm, rain, mist':'heavy', 'Light rain':'light', 'Rain':'med',
 'Heavy thunderstorm, rain, mist':'heavy', 'Light thunderstorm, rain, mist':'med',
 'Heavy rain, mist':'heavy', 'Rain, mist':'med', 'Heavy rain':'heavy', 'Light drizzle, mist':'light',
 'In the vicinity thunderstorm, rain':'med', 'Thunderstorm, rain':'heavy', 'Haze':'none',
 'In the vicinity thunderstorm, light rain, mist':'light', 'Light drizzle':'light',
 'In the vicinity thunderstorm, heavy rain, mist':'heavy',
 'In the vicinity thunderstorm, heavy rain':'heavy',
 'In the vicinity thunderstorm, rain, mist':'med', 'Ground fog':'light',
 'Light rain, snow, mist':'heavy', 'Light rain, ice pellets':'heavy',
 'Light rain, snow, ice pellets, mist':'heavy', 'Light rain, ice pellets, mist':'heavy',
 'Light rain, snow':'med', 'Light snow':'heavy', 'Light snow, mist':'heavy',
 'Light ice pellets, rain':'heavy',
 'Light funnel cloud(s) (tornado or water-spout), rain':'heavy', 'Snow, mist':'heavy',
 'Light snow, ice pellets, mist':'heavy', 'Snow, ice pellets, mist':'heavy',
 'Light funnel cloud(s) (tornado or water-spout), mist':'heavy',
 'Light ice pellets, mist':'heavy', 'Ice pellets, rain':'heavy'})
df = df.fillna('none')
df.groupby('Weather').mean()

,Unnamed: 0,EndZoneID,StartZoneID,TollRate,Temperature,Humidity
Weather,,,,,,
heavy,1.117865e+06,3169.890155,3159.953729,0.721727,15.684190,80.543880
light,1.126372e+06,3169.961615,3160.006985,0.898716,17.130057,83.305298
med,1.129698e+06,3169.994645,3160.035192,1.322756,16.397712,85.313595
none,1.142565e+06,3169.944639,3159.998688,1.200496,20.462790,62.113565


In [4]:
#engineer those features
df = df.drop(['Wind Direction', 'Cloud Cover', 'Unnamed: 0', 'Visibility', 'Pressure', 'Weather Desc'], axis=1)
#df = pd.get_dummies(data=df, columns=['Weather Desc'])

In [5]:
df['TollRate'].mean()

1.1697027077374886

In [6]:
df["date"] = df.StartDateTime.apply(lambda x : x.split()[0])
df["hour"] = df.StartDateTime.apply(lambda x : x.split()[1].split(":")[0])
df["minute"] = df.StartDateTime.apply(lambda x : x.split()[1].split(":")[1])
df["weekday"] = df.date.apply(lambda dateString : calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])
df["month"] = df.date.apply(lambda dateString : calendar.month_name[datetime.strptime(dateString,"%Y-%m-%d").month])

df = df.drop(columns=['StartDateTime', 'EndDateTime', 'date'])

df = pd.get_dummies(data=df, columns=['weekday', 'month', 'Direction', 'Weather'])

#df["season"] = df.season.map({1: "Spring", 2 : "Summer", 3 : "Fall", 4 :"Winter" })
df.head()

,EndZoneID,StartZoneID,TollRate,Temperature,Humidity,Wind Speed,hour,minute,weekday_Friday,weekday_Monday,...,month_May,month_November,month_October,month_September,Direction_E,Direction_W,Weather_heavy,Weather_light,Weather_med,Weather_none
0,3120,3100,0.0,24.0,57.0,4,15,18,0,0,...,0,0,0,0,1,0,0,0,0,1
1,3130,3100,0.0,24.0,57.0,4,15,18,0,0,...,0,0,0,0,1,0,0,0,0,1
2,3230,3210,0.0,24.0,57.0,4,15,18,0,0,...,0,0,0,0,0,1,0,0,0,1
3,3110,3100,0.0,24.0,57.0,4,15,18,0,0,...,0,0,0,0,1,0,0,0,0,1
4,3100,3100,0.0,24.0,57.0,4,15,18,0,0,...,0,0,0,0,1,0,0,0,0,1


In [7]:

#pd.set_option('display.max_columns', 50)
df.shape

(2281425, 33)

In [8]:
df['TollRate'].unique()

array([ 0.  ,  1.  ,  1.5 ,  2.  ,  0.5 ,  3.  ,  8.5 ,  1.75, 10.25,
       11.5 ,  2.5 , 17.5 , 16.25,  1.25,  6.5 , 14.5 ,  7.75,  9.  ,
        3.5 , 11.  , 12.25, 12.75,  7.5 ,  8.75,  0.75, 10.75,  9.75,
        6.  ,  7.  ,  4.75,  9.5 ,  4.25,  6.25,  8.  ,  3.75,  7.25,
        6.75,  5.75,  4.5 ,  9.25,  4.  , 10.5 ,  5.5 ,  5.  ,  2.75,
        3.25,  5.25, 12.  , 11.25, 12.5 ,  2.25, 10.  , 15.75, 16.5 ,
       15.5 , 17.25, 16.  , 14.25,  8.25, 13.25, 13.5 , 15.  , 11.75,
       14.75, 13.  , 14.  , 13.75, 18.25, 17.  , 16.75, 18.  , 22.25,
       21.25, 21.5 , 18.5 , 26.  , 26.75, 19.  , 24.5 , 20.5 , 25.25,
       20.  , 21.  , 19.25, 24.25, 19.75, 20.75, 15.25, 21.75, 22.5 ,
       28.5 , 27.75, 25.  , 24.  , 23.25, 18.75, 19.5 , 25.5 , 26.25,
       30.5 , 31.75, 28.25, 35.25, 36.  , 29.  , 20.25, 31.5 , 30.75,
       23.75, 27.  , 29.25, 34.5 , 31.  , 26.5 , 30.  , 33.75, 34.25,
       29.75, 24.75, 17.75, 32.  , 31.25, 27.25, 27.5 , 38.  , 37.25,
       23.  , 36.5 ,

In [9]:
df = df.dropna()
df = df.replace('none', 0)

In [10]:
from sklearn.model_selection import train_test_split
y = df['TollRate']
X = df.drop(columns=['TollRate'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1, max_iter=1000)
svr_lin = SVR(kernel='linear', C=100, gamma='auto', max_iter=1000)
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1, max_iter=1000)




# #############################################################################
# Look at the results
lw = 2

svrs = [svr_rbf, svr_lin, svr_poly]
for svr in svrs:
    svr = svr.fit(X_train, y_train)
    pred = svr.predict(X_test)
    print(mean_absolute_error(pred, y_test))




In [30]:
from sklearn.linear_model import LinearRegression
#from sklearn import linear_model
model = LinearRegression()
#model = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)

model = model.fit(X_train, y_train)
pred = model.predict(X_test)

ValueError: shapes (752871,33) and (32,) not aligned: 33 (dim 1) != 32 (dim 0)

In [15]:
i = 0
for i in range(len(pred)):
    if pred[i] < 0:
        pred[i] = 0

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pred, y_test)

1.7326908995917907

In [17]:
pred

array([0.        , 2.52134514, 0.42486191, ..., 0.        , 0.        ,
       1.54968834])

In [20]:
X_test['Prediction'] = pred
X_test.groupby('hour').mean()
    

,EndZoneID,StartZoneID,Temperature,Humidity,Wind Speed,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,...,month_November,month_October,month_September,Direction_E,Direction_W,Weather_heavy,Weather_light,Weather_med,Weather_none,Prediction
hour,,,,,,,,,,,,,,,,,,,,,
00,3169.691490,3159.746923,14.099320,72.651541,3.037923,0.156510,0.162818,0.169891,0.148253,0.118128,...,0.116140,0.000000,0.000000,0.501300,0.498700,0.019459,0.132999,0.004205,0.843337,0.906849
01,3170.205825,3160.268608,21.698382,77.575987,2.587638,0.175275,0.176440,0.184725,0.175793,0.095146,...,0.000000,0.000000,0.000000,0.495793,0.504207,0.000000,0.104207,0.000000,0.895793,0.621856
02,3169.750276,3159.892975,21.567315,77.155610,2.945978,0.190521,0.173278,0.167077,0.185509,0.087743,...,0.000000,0.000000,0.000000,0.498598,0.501402,0.000000,0.102353,0.000000,0.897647,0.616228
03,3168.697563,3158.931948,20.961113,78.815821,2.615528,0.190438,0.176854,0.177920,0.149288,0.090691,...,0.000000,0.000000,0.000000,0.509122,0.490878,0.010121,0.178852,0.000000,0.811027,0.635720
04,3170.316625,3160.397661,20.663993,78.535255,2.912114,0.185798,0.185297,0.177527,0.157895,0.093734,...,0.000000,0.000000,0.000000,0.496575,0.503425,0.015038,0.071763,0.000000,0.913200,0.619253
05,3170.306314,3160.412097,20.265494,79.777078,2.761892,0.173665,0.168278,0.180025,0.181972,0.091959,...,0.000000,0.000000,0.000000,0.495100,0.504900,0.000000,0.009021,0.000000,0.990979,0.575504
06,3170.615069,3160.764992,19.994618,80.608598,2.624616,0.174846,0.177537,0.183175,0.174846,0.084380,...,0.000000,0.000000,0.000000,0.491735,0.508265,0.000000,0.105715,0.000000,0.894285,0.570807
07,3169.892597,3159.940545,20.616673,79.530367,2.308273,0.177151,0.180540,0.185079,0.157269,0.091996,...,0.000000,0.000000,0.000000,0.499808,0.500192,0.000000,0.249009,0.000000,0.750991,0.633947
08,3170.322478,3160.390269,21.370058,73.986885,2.872276,0.172010,0.186011,0.192980,0.159465,0.090788,...,0.000570,0.011087,0.007159,0.495248,0.504752,0.000824,0.032375,0.000697,0.966105,0.736100


In [21]:
df.groupby('hour').mean()

,EndZoneID,StartZoneID,TollRate,Temperature,Humidity,Wind Speed,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,...,month_May,month_November,month_October,month_September,Direction_E,Direction_W,Weather_heavy,Weather_light,Weather_med,Weather_none
hour,,,,,,,,,,,,,,,,,,,,,
00,3169.870831,3159.935416,0.000000,14.086735,72.570311,3.049599,0.156264,0.162874,0.169131,0.148292,...,0.000000,0.117059,0.000000,0.000000,0.500000,0.500000,0.018669,0.133962,0.004037,0.843332
01,3169.778912,3159.889456,0.000000,21.678146,77.624490,2.572704,0.173214,0.175000,0.183588,0.177806,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.100170,0.000000,0.899830
02,3169.698997,3159.849498,0.000000,21.554181,77.277815,2.956410,0.191862,0.174136,0.170123,0.184058,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.106355,0.000000,0.893645
03,3169.588101,3159.794050,0.000000,20.949657,78.879423,2.605351,0.195036,0.182538,0.173209,0.149269,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.009681,0.178666,0.000000,0.811653
04,3169.957811,3159.978905,0.000000,20.679138,78.524259,2.919951,0.187188,0.180526,0.173532,0.162429,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.013878,0.069390,0.000000,0.916731
05,3169.775647,3159.887823,0.000000,20.242422,79.700891,2.757236,0.177941,0.168608,0.177342,0.181110,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.008392,0.000000,0.991608
06,3169.775720,3159.887860,0.000000,20.022683,80.447116,2.633591,0.174667,0.177555,0.183332,0.174497,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.105004,0.000000,0.894996
07,3169.779661,3159.889831,0.000000,20.631186,79.580424,2.299661,0.179831,0.182119,0.186610,0.156186,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.250000,0.000000,0.750000
08,3169.790140,3159.895070,0.000000,21.340190,73.933461,2.869254,0.174634,0.184377,0.190207,0.161143,...,0.008328,0.000416,0.010826,0.007079,0.500000,0.500000,0.001249,0.032062,0.000833,0.965856


In [27]:
i = 0
coefficients = model.coef_
for col in X_train.columns.values:
    print(col + ':   ' + str(coefficients[i]))
    i += 1

EndZoneID:   0.0594394185123674
StartZoneID:   -0.058415001572138685
Temperature:   0.04294272100698906
Humidity:   -0.010480536963757468
Wind Speed:   -0.009999383730163822
hour:   0.008725151616659346
minute:   0.0011002726573278766
weekday_Friday:   826048292.195087
weekday_Monday:   826048292.765532
weekday_Saturday:   826048291.1551989
weekday_Sunday:   826048291.1469879
weekday_Thursday:   826048292.9967602
weekday_Tuesday:   826048293.1111877
weekday_Wednesday:   826048293.0158831
month_April:   -84346385.57397516
month_August:   -84346386.32468411
month_December:   -84346385.18533967
month_February:   -84346385.3557664
month_January:   -84346385.6928452
month_July:   -84346386.51640804
month_June:   -84346386.74660325
month_March:   -84346385.31092483
month_May:   -84346385.4362424
month_November:   -84346385.33505942
month_October:   -84346385.35499956
month_September:   -84346385.81186981
Direction_E:   5975004185.482998
Direction_W:   5975004184.828218
Weather_heavy:   -1671

In [29]:
model.score(X_test.drop('Prediction', axis=1), y_test)

0.10294963576494377

In [35]:
from sklearn.linear_model import ARDRegression, LinearRegression

degree = 10
clf_poly = ARDRegression(threshold_lambda=.1)
clf_poly.fit(X_train, y_train)
pred = clf_poly.predict(X_test)
mean_absolute_error(pred, y_test)

MemoryError: 

In [ ]:
df.head()

In [ ]:
df.groupby(['StartZoneID', 'EndZoneID']).mean()

In [ ]:
filtered = df[df.StartZoneID == 3100]
filtered = filtered[filtered.EndZoneID == 3130]
#filtered = filtered[filtered.hour == 18]
#filtered = filtered.drop_duplicates()
#filtered.head(500)


In [ ]:
filtered.groupby('month').mean()

In [ ]:
labels = ["low", "low/med", "medium", "med/high", "high"]


filtered['TempBinned'] = pd.cut(filtered['Temperature'], bins=5)
filtered.head()

In [ ]:
filtered['Humidity'] = pd.cut(df['Humidity'], bins=8)
filtered.groupby('Humidity').mean()

In [ ]:
filtered.head(3)

In [ ]:
filtered.groupby('Temperature').mean()

In [ ]:
filtered['Temperature'].value_counts()

In [ ]:
df.groupby('hour').mean()

In [ ]:
filtered['Visibility'] = pd.cut(df['Visibility'], bins=8)
filtered.groupby('Visibility').mean()

In [ ]:
filtered['Visibility'].value_counts()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
weather = pd.read_csv('csv_files/weather.csv')
weather.head()

In [ ]:
weather["hour"] = weather.Time.apply(lambda x : x.split()[1].split(":")[0])
weather.head()